## Homework-3: MNIST Classification with ConvNet

### **Deadline: 2021.04.06 23:59:00 **

### In this homework, you need to
- #### implement the forward and backward functions for ConvLayer (`layers/conv_layer.py`)
- #### implement the forward and backward functions for PoolingLayer (`layers/pooling_layer.py`)
- #### implement the forward and backward functions for DropoutLayer (`layers/dropout_layer.py`)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

from network import Network
from solver import train, test
from plot import plot_loss_and_acc

C:\Users\cascara\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\cascara\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\cascara\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\casc

## Load MNIST Dataset
We use tensorflow tools to load dataset for convenience.

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() 

In [3]:
def decode_image(image):
    # Normalize from [0, 255.] to [0., 1.0], and then subtract by the mean value
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, [1, 28, 28])
    image = image / 255.0
    image = image - tf.reduce_mean(image)
    return image

def decode_label(label):
    # Encode label with one-hot encoding
    return tf.one_hot(label, depth=10)

In [4]:
# Data Preprocessing
x_train = tf.data.Dataset.from_tensor_slices(x_train).map(decode_image)
y_train = tf.data.Dataset.from_tensor_slices(y_train).map(decode_label)
data_train = tf.data.Dataset.zip((x_train, y_train))
x_test = tf.data.Dataset.from_tensor_slices(x_test).map(decode_image)
y_test = tf.data.Dataset.from_tensor_slices(y_test).map(decode_label)
data_test = tf.data.Dataset.zip((x_test, y_test))

## Set Hyperparameters
You can modify hyperparameters by yourself.

In [5]:
batch_size = 100
max_epoch = 10
init_std = 0.01

learning_rate = 0.001
weight_decay = 0.005

disp_freq = 50

## Criterion and Optimizer

In [7]:
from criterion import SoftmaxCrossEntropyLossLayer
from optimizer import SGD

criterion = SoftmaxCrossEntropyLossLayer()
sgd = SGD(learning_rate, weight_decay)

## ConvNet

In [20]:
foo=np.random.random((2,4,5))
foo.sum(axis=(0,2)).shape

(4,)

In [61]:
a=np.random.randint(2,3,k)
k=9
b=np.random.randint(2,10,k)
pad=(k-1)//2
print(signal.convolve(a,b,mode='full')[pad:-pad])
print(signal.convolve(a,b,mode='same')[pad])
print(signal.convolve(a,b,mode='same'))
print(signal.convolve(a,b,mode='valid'))

[ 68  84 100 106 120 104  86  82  70]
120
[ 68  84 100 106 120 104  86  82  70]
[120]


In [33]:
from scipy import signal 
height=5
width=4
batch_size=3
channels=4
kernel=3
inputs=4
a = np.random.randint(0,5,(height, width,channels, batch_size))
b = np.random.randint(0,5,( kernel,kernel,channels, batch_size))
c = np.random.rand(width, height)

output=signal.convolve(a,b,mode='valid')
print(output.shape)
outputs=np.zeros((batch_size,height-kernel+1,width-kernel+1,1))
for i in range(batch_size):
    outputs[i]=signal.convolve(a[:,:,:,i],b[:,:,:,i],mode='valid')
print(outputs.shape)
# print(output)

print(outputs.sum(axis=0)-output[:,:,:,0])


(3, 2, 1, 1)
(3, 3, 2, 1)
[[[  -1.]
  [-103.]]

 [[  11.]
  [  28.]]

 [[   4.]
  [ -23.]]]


In [25]:
print(output)


[[[[24.12249346]]]


 [[[24.38835166]]]]


In [24]:
print(outputs.sum(axis=0))


[[[25.15004997]]

 [[25.38613493]]]


In [17]:
import numpy as np
height=4
width=3
batch_size=3
channels=4
kernel=3
inputs=4
a = np.random.rand(height, width,channels, batch_size)
b = np.random.rand( kernel,kernel,channels, channels)
c = np.random.rand(width, height)

# zz = np.matmul( a, c)
# print("\n*************** numpy 输出 {} ****************".format(xx.shape))

# print(zz)

# for ai in a:
#     print(np.dot(ai,c))
# err_max2 = np.amax(np.absolute(np.subtract(out, xx)))
# print("\ntf与numpy误差:{}".format(err_max2))
from scipy import signal

res=signal.convolve(a,np.flip(b[:,:,:,0][:,:,:,None],(0,1,2)),mode='valid')
res2=signal.convolve(a[:,:,:,None,:],np.flip(b[:,:,:,:,None],(0,1,2)),mode='valid')
print(res2.shape)
# res=signal.convolve(a,np.flip(b[:,:,:,None],(0,1,2)),mode='valid')
res.shape

ValueError: For 'valid' mode, one must be at least as large as the other in every dimension

In [16]:
print(a[:,:,:,:,None].shape)
print(a[:,:,:,None,:].shape)

(4, 3, 4, 3, 1)
(4, 3, 4, 1, 3)


In [12]:
b.shape

(3, 3, 4, 4)

In [13]:
b[:,:,:,0].shape

(3, 3, 4)

In [7]:
res

array([[[[34.98327296, 34.46606456, 31.95878098]]],


       [[[32.00600476, 35.5741041 , 31.47969642]]]])

In [4]:
res

array([[[[35.22705346, 36.1332452 , 28.24937155]],

        [[34.50411452, 32.36339304, 30.94247298]],

        [[32.11687527, 31.04864529, 30.1755373 ]],

        ...,

        [[30.72137001, 31.25542539, 27.54375958]],

        [[29.62255746, 29.22852617, 30.41488096]],

        [[31.86715658, 29.21253837, 31.89024996]]],


       [[[35.55724162, 32.74041396, 30.50603999]],

        [[31.81656379, 31.22564367, 30.41859773]],

        [[28.84225168, 28.51268174, 28.49778645]],

        ...,

        [[31.12637934, 30.95240164, 29.12769874]],

        [[35.46195581, 29.31269039, 30.54670851]],

        [[32.24688127, 32.06875635, 31.24343029]]],


       [[[28.6105157 , 30.46160829, 30.97962276]],

        [[28.09282702, 29.14577136, 31.90656051]],

        [[29.50653614, 27.75676068, 30.50173261]],

        ...,

        [[30.29093008, 30.11218732, 29.31762839]],

        [[31.54745673, 30.35729104, 29.32911757]],

        [[33.01866191, 28.13444301, 28.44027122]]],


       ...,


  

In [1]:
height=24
width=20
batch_size=3
channels=1
kernel=3
inputs=4
a = np.random.rand(channels, height, width,batch_size)
b = np.random.rand( kernel,kernel,channels)
c = np.random.rand(width, height)

# zz = np.matmul( a, c)
# print("\n*************** numpy 输出 {} ****************".format(xx.shape))

# print(zz)

# for ai in a:
#     print(np.dot(ai,c))
# err_max2 = np.amax(np.absolute(np.subtract(out, xx)))
# print("\ntf与numpy误差:{}".format(err_max2))
from scipy import signal

res=signal.convolve(a,b[:,:,:,None])

NameError: name 'np' is not defined

In [ ]:
res[:,:,0,:].shape

(22, 18, 3)

In [114]:
c

array([[0.65983091, 0.76265193, 0.83145357],
       [0.99746009, 0.46267084, 0.91773776],
       [0.34008609, 0.43997009, 0.55308495],
       [0.62878291, 0.73104422, 0.25967142],
       [0.55489938, 0.31974324, 0.36953437],
       [0.32262288, 0.31330695, 0.76618513]])

In [121]:
height=6
width=3
batch_size=3
stride=1
kernel=3
c=np.random.random([height,width])
k=np.random.random([kernel,kernel])#,width])


strides=(width*stride,stride,width,1)
strides2=(c.strides[0]*k.shape[0],k.strides[0],c.strides[0],c.strides[1],)#c.strides[2],c.strides[1],c.strides[2])
shape = ( int(height / kernel), int(width / kernel) , kernel, kernel)
print(c)
np.lib.stride_tricks.as_strided(c,shape=shape,strides=strides2)

[[0.15482158 0.72507722 0.08870157]
 [0.82229885 0.11580479 0.3044201 ]
 [0.8487845  0.30354088 0.45042539]
 [0.06588872 0.23639724 0.8291089 ]
 [0.56700259 0.32220286 0.12749653]
 [0.47881041 0.44147604 0.05963717]]


array([[[[0.15482158, 0.72507722, 0.08870157],
         [0.82229885, 0.11580479, 0.3044201 ],
         [0.8487845 , 0.30354088, 0.45042539]]],


       [[[0.06588872, 0.23639724, 0.8291089 ],
         [0.56700259, 0.32220286, 0.12749653],
         [0.47881041, 0.44147604, 0.05963717]]]])

In [119]:
c.strides[0]*k.shape[0]

72

In [117]:

k.strides

(24, 8)

In [118]:
c.strides

(24, 8)

In [94]:
res=np.lib.stride_tricks.as_strided(c,shape=shape,strides=strides2)


In [1]:
import numpy as np

In [54]:
import numpy as np
m=np.zeros((2,3))
m[0,2]=1
m[1,1]=1
n=np.random.randint(1,5,(2))
print(m)
print(n)
print(np.multiply(m,n[:,None]))

[[0. 0. 1.]
 [0. 1. 0.]]
[2 1]
[[0. 0. 2.]
 [0. 1. 0.]]


In [55]:
t=np.random.randint(2,19,size=(3,2,3,3))

In [33]:
t[0,0,5]

18

In [58]:
t

array([[[[18,  4, 16],
         [ 7,  6,  5],
         [ 8, 13,  9]],

        [[ 3, 13, 14],
         [ 6,  3, 14],
         [ 7,  6, 11]]],


       [[[ 6, 14, 16],
         [12,  5, 11],
         [10, 16,  5]],

        [[ 8, 16, 13],
         [ 8,  2, 10],
         [13,  5,  6]]],


       [[[16, 14, 18],
         [10, 18, 11],
         [13,  6, 14]],

        [[ 5,  9, 11],
         [18, 14, 17],
         [15, 18, 17]]]])

In [56]:
t_reshape=t.reshape((6,9))
t_arg=t_reshape.argmax(axis=1)#.reshape((3,2,9))

In [62]:
np.reshape(t_reshape,t.shape)

array([[[[ 0,  4, 16],
         [ 7,  6,  5],
         [ 8, 13,  9]],

        [[ 3, 13,  0],
         [ 6,  3, 14],
         [ 7,  6, 11]]],


       [[[ 6, 14,  0],
         [12,  5, 11],
         [10, 16,  5]],

        [[ 8,  0, 13],
         [ 8,  2, 10],
         [13,  5,  6]]],


       [[[16, 14,  0],
         [10, 18, 11],
         [13,  6, 14]],

        [[ 5,  9, 11],
         [ 0, 14, 17],
         [15, 18, 17]]]])

In [70]:
import tensorflow as tf
label=2
tf.one_hot(label, depth=10)

<tf.Tensor 'one_hot_2:0' shape=(10,) dtype=float32>

In [71]:
def decode_image(image):
    # Normalize from [0, 255.] to [0., 1.0], and then subtract by the mean value
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, [1, 28, 28])
    # image = image / 255.0
    image = image - tf.reduce_mean(image)
    return image

def decode_label(label):
    # Encode label with one-hot encoding
    return tf.one_hot(label, depth=10)

In [72]:
train_x = decode_image(np.random.random((28,28)))

In [74]:
np.array(train_x).shape

()

In [75]:
train_x.shape

TensorShape([Dimension(1), Dimension(28), Dimension(28)])

In [67]:
t[:,:,:,:,None].shape

(3, 2, 3, 3, 1)

In [57]:
t_reshape

array([[18,  4, 16,  7,  6,  5,  8, 13,  9],
       [ 3, 13, 14,  6,  3, 14,  7,  6, 11],
       [ 6, 14, 16, 12,  5, 11, 10, 16,  5],
       [ 8, 16, 13,  8,  2, 10, 13,  5,  6],
       [16, 14, 18, 10, 18, 11, 13,  6, 14],
       [ 5,  9, 11, 18, 14, 17, 15, 18, 17]])

In [59]:
np.indices(t_arg.shape)

array([[0, 1, 2, 3, 4, 5]])

In [60]:
t_reshape[np.indices(t_arg.shape),t_arg]=0

In [61]:
t_reshape

array([[ 0,  4, 16,  7,  6,  5,  8, 13,  9],
       [ 3, 13,  0,  6,  3, 14,  7,  6, 11],
       [ 6, 14,  0, 12,  5, 11, 10, 16,  5],
       [ 8,  0, 13,  8,  2, 10, 13,  5,  6],
       [16, 14,  0, 10, 18, 11, 13,  6, 14],
       [ 5,  9, 11,  0, 14, 17, 15, 18, 17]])

In [46]:
t_reshape[:,t_arg]

array([[18, 18,  6, 18,  6,  6],
       [17, 17,  4, 17,  4,  4],
       [10, 10, 17, 10, 17, 17],
       [16, 16, 13, 16, 13, 13],
       [12, 12, 18, 12, 18, 18],
       [ 4,  4, 18,  4, 18, 18]])

In [36]:
t.argmax(axis=0)

array([[2, 2, 0, 0, 0, 0, 0, 0, 0],
       [2, 2, 2, 0, 2, 2, 0, 2, 0]], dtype=int64)

In [17]:
t[:,t.argmax(axis=1)]

array([[[[8]],

        [[6]],

        [[8]]],


       [[[7]],

        [[7]],

        [[7]]],


       [[[7]],

        [[2]],

        [[7]]]])

In [9]:
t[:][t==t.max(axis=(1,2))]

IndexError: boolean index did not match indexed array along dimension 2; dimension is 1 but corresponding boolean dimension is 3

In [95]:
res.shape

(3, 2, 3, 3)

In [97]:
res

array([[[[0.07918448, 0.44948342, 0.66026626],
         [0.12726747, 0.9503967 , 0.88146213],
         [0.08915318, 0.03531358, 0.8685482 ]],

        [[0.44948342, 0.66026626, 0.50844383],
         [0.9503967 , 0.88146213, 0.73446691],
         [0.03531358, 0.8685482 , 0.62420618]]],


       [[[0.12726747, 0.9503967 , 0.88146213],
         [0.08915318, 0.03531358, 0.8685482 ],
         [0.8213609 , 0.58456966, 0.88256334]],

        [[0.9503967 , 0.88146213, 0.73446691],
         [0.03531358, 0.8685482 , 0.62420618],
         [0.58456966, 0.88256334, 0.59978196]]],


       [[[0.08915318, 0.03531358, 0.8685482 ],
         [0.8213609 , 0.58456966, 0.88256334],
         [0.61682261, 0.94946307, 0.24180738]],

        [[0.03531358, 0.8685482 , 0.62420618],
         [0.58456966, 0.88256334, 0.59978196],
         [0.94946307, 0.24180738, 0.8887776 ]]]])

In [98]:
res.max(axis=(-2,-1))

array([[0.9503967 , 0.9503967 ],
       [0.9503967 , 0.9503967 ],
       [0.94946307, 0.94946307]])

In [ ]:
height=5
width=4
batch_size=3
stride=1
kernel=3

c=np.random.random([batch_size,height,width])

strides=(width*stride,stride,width,1)
strides2=(c.strides[0],c.strides[1],c.strides[2],c.strides[1],c.strides[2])
shape = (batch_size, height - kernel + 1, width - kernel + 1, kernel, kernel)
np.lib.stride_tricks.as_strided(c,shape=shape,strides=strides2)

In [93]:
c.da

array([[0.07918448, 0.44948342, 0.66026626, 0.50844383],
       [0.12726747, 0.9503967 , 0.88146213, 0.73446691],
       [0.08915318, 0.03531358, 0.8685482 , 0.62420618],
       [0.8213609 , 0.58456966, 0.88256334, 0.59978196],
       [0.61682261, 0.94946307, 0.24180738, 0.8887776 ]])

In [85]:
c=np.random.random([height,width])
c.strides

(32, 8)

In [91]:
from layers import FCLayer, ReLULayer, ConvLayer, MaxPoolingLayer, ReshapeLayer

convNet = Network()
convNet.add(ConvLayer(1, 8, 3, 1))
convNet.add(ReLULayer())
convNet.add(MaxPoolingLayer(2, 0))
convNet.add(ConvLayer(8, 16, 3, 1))
convNet.add(ReLULayer())
convNet.add(MaxPoolingLayer(2, 0))
convNet.add(ReshapeLayer((batch_size, 16, 7, 7), (batch_size, 784)))
convNet.add(FCLayer(784, 128))
convNet.add(ReLULayer())
convNet.add(FCLayer(128, 10))

ImportError: cannot import name 'ConvLayer' from 'layers' (c:\Users\cascara\Desktop\code\2021spring\patternRecognition\homework-12345\homework3\layers\__init__.py)

In [ ]:
# Train
convNet.is_training = True
convNet, conv_loss, conv_acc = train(convNet, criterion, sgd, data_train, max_epoch, batch_size, disp_freq)

In [ ]:
# Test
convNet.is_training = False
test(convNet, criterion, data_test, batch_size, disp_freq)

## Plot

In [ ]:
plot_loss_and_acc({'ConvNet': [conv_loss, conv_acc]})

### ~~You have finished homework3, congratulations!~~  

**Next, according to the requirements (4):**
### **You need to implement the Dropout layer and train the network again.**

In [ ]:
from layers import DropoutLayer

# build your network

# training

# testing